In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import patches
from scipy import interpolate, spatial
from tqdm.auto import tqdm

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
RNG = np.random.default_rng(42)

Utility functions for visualization

In [ ]:
def config_rcparams(reset: bool = False) -> None:
    """Set seaborn rcParams for consistent plotting style.

    Args:
        reset (bool, optional): If True, reset to seaborn defaults.

    Returns:
        None

    """
    if reset:
        sns.reset_defaults()
    else:
        sns.set(
            context="notebook",
            style="white",
            rc={
                "xtick.bottom": True,
                "xtick.color": "black",
                "xtick.direction": "in",
                "ytick.direction": "in",
            },
        )
        sns.set_palette("colorblind")

In [ ]:
def set_3d_params(ax: plt.Axes, aspect: tuple[int, int, int] = (1, 1, 1)) -> plt.Axes:
    """Configure 3D plot parameters.

    Args:
        ax (plt.Axes): 3D axes to configure.
        aspect (tuple, optional): Aspect ratio for x, y, z axes.

    Returns:
        plt.Axes: Configured 3D axes.

    """
    ax.set(xlabel="x", ylabel="y", zlabel="z")
    ax.xaxis.set_major_locator(plt.MaxNLocator(3))
    ax.yaxis.set_major_locator(plt.MaxNLocator(3))
    ax.zaxis.set_major_locator(plt.MaxNLocator(3))
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.set_box_aspect(aspect)
    return ax

# Input data

In [ ]:
def generate_2d_gaussian(
    x: float | np.ndarray,
    y: float | np.ndarray,
    a: float = 1,
    x0: float = 0,
    y0: float = 0,
    theta_x: float = 1,
    theta_y: float = 1,
) -> float | np.ndarray:
    """Generate a 2D Gaussian function.

    Parameters
    ----------
    x : float of numpy.ndarray
        Spatial coordinate(s), x-direction
    y : float of numpy.ndarray
        Spatial coordinate(s), y-direction
    a : float, optional
        Amplitude
    x0 : float, optional
        Center of the blob, x-direction
    y0 : float, optional
        Center of the blob, y-direction
    theta_x : float, optional
        Spread of the blob, x-direction
    theta_y : float, optional
        Spread of the blob, y-direction

    Returns
    -------
    float or numpy.ndarray
        Value(s) of the Guassian function, z-direction

    """
    return a * np.exp(
        -((x - x0) ** 2) / (2 * theta_x**2) - (y - y0) ** 2 / (2 * theta_y**2),
    )

In [ ]:
# Generate the 2D Gaussian
x = np.linspace(-1, 1, 51)
y = np.linspace(-1, 1, 51)
X, Y = np.meshgrid(x, y)
Z = generate_2d_gaussian(X, Y, a=2, theta_x=0.3, theta_y=0.3)

In [ ]:
# Plot the surface
config_rcparams()
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax = set_3d_params(ax)
surf = ax.plot_surface(X, Y, Z, lw=0, cstride=1, rstride=1, antialiased=False)

In [ ]:
def estimate_normals(points: np.ndarray, k: int) -> np.ndarray:
    """Estimate point normals using PCA on local neighborhoods.

    Args:
        points (numpy.ndarray): Nx3 array of point coordinates.
        k (int): Number of nearest neighbors to consider.

    Returns:
        numpy.ndarray: Nx3 array of estimated normals.

    """
    # Create a kd-tree for quick nearest-neighbor lookup
    tree = spatial.KDTree(points)
    n = np.empty_like(points)
    for i, p in enumerate(points):
        # Extract the local neighborhood
        _, idx = tree.query([p], k=k, eps=0.1, workers=-1)
        nbhd = points[idx.flatten()]

        # Extract an eigenvector with smallest associated eigenvalue
        x = nbhd.copy()
        x = x - np.mean(x, axis=0)
        c = x.T @ x
        u, s, _ = np.linalg.svd(c)
        n[i, :] = u[:, np.argmin(s)]
    return n

In [ ]:
def orient_normals(
    points: np.ndarray,
    normals: np.ndarray,
    k: int | None = None,
    convex: bool = False,
) -> np.ndarray:
    """Orient point normals consistently.

    Args:
        points (numpy.ndarray): Nx3 array of point coordinates.
        normals (numpy.ndarray): Nx3 array of point normals.
        k (int, optional): Number of neighbors for orientation consistency.
        convex (bool, optional): If True, orient normals to point outward from centroid.

    Returns:
        numpy.ndarray: Nx3 array of oriented normals.

    """
    if convex:
        center = np.mean(points, axis=0)
        for i in range(points.shape[0]):
            pi = points[i, :] - center
            ni = normals[i]
            angle = np.arccos(np.clip(np.dot(ni, pi), -1.0, 1.0))
            if (angle > np.pi / 2) or (angle < -np.pi / 2):
                normals[i] = -ni
        return normals
    try:
        import open3d as o3d
    except ModuleNotFoundError as e:
        print(e, "install 'open3d' before proceeding", sep=", ")
    else:
        o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel(3))
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.normals = o3d.utility.Vector3dVector(normals)
    pcd.orient_normals_consistent_tangent_plane(k)
    return np.asarray(pcd.normals)

In [ ]:
# Create the point cloud and generate a unit normal at each point
points = np.c_[X.ravel(), Y.ravel(), Z.ravel()]
normals = estimate_normals(points, k=20)
normals = orient_normals(points, normals, k=20)

In [ ]:
# Plot the surface with normals
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax = set_3d_params(ax)
surf = ax.plot_surface(X, Y, Z, lw=0, cstride=1, rstride=1, antialiased=False)
q = ax.quiver(
    *points.T,
    *normals.T,
    color="k",
    lw=0.5,
    length=0.25,
    arrow_length_ratio=0.15,
)

# Assesment of location of a query point relative to the point cloud

## RBF interpolation

The first method in this notebook interpolates the surface points in 3D space by using RBF.

Herein, the [`polatory`](https://github.com/polatory/polatory) package, a fast and memory-efficient framework written in C++, is used. This package implements the approach proposed in Carr et al. "[Reconstruction and representation of 3D objects with radial basis functions](https://doi.org/10.1145/383259.383266)," in *Computer Graphics SIGGRAPH 2001 proceedings*, pp. 67-76, 2001.

This approach is divided into 4 simple steps:

**Step 1** &ensp; Define the query point, $p$

**Step 2** &ensp; Create the signed-distance function and sample points and values for interpolation purposes

**Step 3** &ensp; Interpolate sampled points by using RBF (bi-harmonic kernel)

**Step 4** &ensp; Evaluate the interpolant at $p$; if the value is positive, the point is located out of the point cloud

In [ ]:
try:
    import polatory
except ModuleNotFoundError as e:
    print(e, "install 'polatory' before proceeding", sep=", ")
else:
    # Step 1
    point_out = np.array([-1, -1, 1])  # out of the point cloud

    # Step 2
    pairwise_distance = spatial.distance.pdist(points)
    min_distance = np.min(pairwise_distance)
    max_distance = np.max(pairwise_distance)
    sdf = polatory.SdfDataGenerator(points, normals, min_distance, max_distance)
    sdf_points, sdf_values = sdf.sdf_points, sdf.sdf_values
    mask = polatory.DistanceFilter(sdf_points, 1e-4).filtered_indices
    sdf_points, sdf_values = sdf_points[mask, ...], sdf_values[mask]

    # Step 3
    rbf = polatory.Biharmonic3D([1.0])
    model = polatory.Model(rbf, poly_dimension=2, poly_degree=1)
    interp = polatory.Interpolant(model)
    interp.fit(sdf_points, sdf_values, absolute_tolerance=1e-4)

    # Step 4
    val = interp.evaluate(point_out)
    if val > 0:
        print("The point is OUT of the Gaussian surface")
    else:
        print("The point is WITHIN the Gaussian surface")

Previously outlined method can be conveniently wrapped in a function and used in a vectorized fashion handling multiple query points at the same time. In the following implementation, RBF interpolation is done by using `SciPy` instead of `polatory` for speed, simplicity, and controlability sake.

In [ ]:
MIN_SIZE = 10  # Minimum number of points required

In [ ]:
def assess_position(
    query_points: np.ndarray,
    evaluation_points: np.ndarray,
    normals: np.ndarray = None,
    k: int | None = None,
) -> np.ndarray:
    """Assess whether query points are inside or outside a surface.

    Args:
        query_points (numpy.ndarray): Mx3 array of query point coordinates.
        evaluation_points (numpy.ndarray): Nx3 array of surface point coordinates.
        normals (numpy.ndarray, optional): Nx3 array of surface point normals.
        k (int, optional): Number of neighbors for normal estimation and orientation

    Returns:
        numpy.ndarray: Array of size M with signed distance values:
          - positive if outside
          - negative if inside.

    """
    # Handle points
    size = evaluation_points.shape[0]
    if size < MIN_SIZE:
        msg = f"Number of points must be > {MIN_SIZE}"
        raise ValueError(msg)  # For robustness

    # Compute normals
    if normals is None:
        if not k:
            k = int(2 * np.log(size))
            k = max(5, min(k, 30))  # Constrain k between 5 and 30
        normals = estimate_normals(evaluation_points, k)
        normals = orient_normals(evaluation_points, normals, k)
    normals = normals / np.linalg.norm(normals, axis=1).reshape(-1, 1)

    # Sample points sampled from the signed distance function
    pairwise_distance = spatial.distance.pdist(evaluation_points)
    min_distance = np.min(pairwise_distance)
    max_distance = np.max(pairwise_distance)
    sdf = polatory.SdfDataGenerator(
        evaluation_points,
        normals,
        min_distance,
        max_distance,
    )

    # Remove points that are too close to each other
    mask = polatory.DistanceFilter(sdf.sdf_points, 1e-4).filtered_indices
    sdf_points = sdf.sdf_points[mask, ...]
    sdf_values = sdf.sdf_values[mask]

    # Interpolate SDF points with RBF, Carr et al. 2001
    interp = interpolate.RBFInterpolator(
        sdf_points,
        sdf_values,
        kernel="linear",  # Biharmonic kernel
        degree=1,
    )
    return interp(np.atleast_2d(query_points))

In [ ]:
# Define query points to test an assessment function
query_points = np.c_[X.ravel(), Y.ravel(), np.ones_like(X).ravel()]

In [ ]:
# Plot query points in 3D
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax.contourf(X, Y, Z, zdir="y", offset=1, levels=1, colors="b")
ax.contourf(X, Y, Z, zdir="x", offset=-1, levels=1, colors="b")
ax.scatter(*query_points.T, fc="orange", ec="k", s=5, lw=0.5)
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# Check if the orange dots are inside or out of the point cloud
try:
    import polatory
except ModuleNotFoundError as e:
    print(e, "install 'polatory' before proceeding", sep=", ")
else:
    # Assess position of query points relative to the surface
    val = assess_position(query_points, points, normals=normals)

    # Show only points that are outside the surface
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection="3d")
    ax.contourf(X, Y, Z, zdir="y", offset=1, levels=1, colors="b")
    ax.contourf(X, Y, Z, zdir="x", offset=-1, levels=1, colors="b")
    ax.scatter(*query_points[val > 0, ...].T, fc="w", ec="k", s=15, lw=0.5)
    ax = set_3d_params(ax)
    ax.view_init(25, -70)

    # Find out the approximate radius where f(x, y) is ~1
    idx = np.where(np.isclose(Z, 1, rtol=1e-2, atol=1e-2))
    r = np.mean(np.sqrt(X[idx] ** 2 + Y[idx] ** 2))

    # Plot only the xy-projection of the outside points
    fig = plt.figure(figsize=(4, 4))
    ax = plt.axes()
    circle = patches.Circle((0, 0), r, fc="none", ec="k")
    ax.add_patch(circle)
    ax.scatter(*query_points[val > 0, :2].T, fc="w", ec="k", s=7, lw=0.5)
    ax.set(xlabel="x", ylabel="y")

## Normal direction

The second method follows 4 very simple steps:

**Step 1** &ensp; Define the query point, $p$

**Step 2** &ensp; Find $k$ points on the point-cloud surface nearest to $p$ 

**Step 3** &ensp; Compute the scalar product between the relative position vector to $p$ from each of the $k$-nearest neighbors and the corresponding unit normal vector, $\mathbf{\hat{n}_i}$:

$$ \lvert \mathbf{p} - \mathbf{x_i} \rvert \cdot {\mathbf{\hat{n}_i}} $$

**Step 4** &ensp; Count the negative vs. positive values obtained in the previous step; if the ratio of the positive numbers is higher compared to the positive numbers, the point is located out of the point cloud

In [ ]:
# Step 1
point_out = np.array([1, -1, 2])  # out of the point cloud

In [ ]:
# Show the point in 3D
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
surf = ax.plot_surface(X, Y, Z, lw=0, cstride=1, rstride=1, antialiased=False)
ax.scatter(*points.T, fc="w", ec="k", s=5, lw=0.5)
ax.scatter(*point_out, fc="orange", ec="k", s=15, lw=0.5)
ax.text(*point_out + np.array([0, 0, 0.2]), f"{point_out}")
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# Step 2
tree = spatial.KDTree(points)
dist, idx = tree.query(point_out, k=20)

In [ ]:
# Show the point in 3D with neighbors
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax.scatter(*np.delete(points, idx, axis=0).T, fc="w", ec="k", s=5, lw=0.5)
ax.scatter(*points[idx, ...].T, fc="green", ec="k", s=15, lw=0.5)
ax.scatter(*point_out, fc="orange", ec="k", s=15, lw=0.5)
ax.text(*point_out + np.array([0, 0, 0.2]), f"{point_out}")
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# Show the point in 3D with neighbors and normals
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax.scatter(*np.delete(points, idx, axis=0).T, fc="w", ec="k", s=5, lw=0.5)
ax.scatter(*points[idx, ...].T, fc="green", ec="k", s=15, lw=0.5)
ax.quiver(
    *points[idx, ...].T,
    *normals[idx, ...].T,
    color="k",
    lw=0.5,
    length=0.5,
    arrow_length_ratio=0.15,
)
ax.scatter(*point_out, fc="orange", ec="k", s=15, lw=0.5)
ax.scatter(0, 0, 0, fc="k", ec="k", s=15, lw=0.5)
ax.quiver(0, 0, 0, *point_out, color="k", lw=1, arrow_length_ratio=0.1)
ax.text(*point_out + np.asarray([0, 0, 0.2]), f"{point_out}")
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# Step 3
prod = np.sum((point_out - points[idx]) * normals[idx], axis=1)

In [ ]:
# Step 4
prob_threshold = 0.5
prob = np.sum(prod > 0) / prod.size
if prob > prob_threshold:
    print(f"The point is OUT of the point cloud ({prob:.2f})")
else:
    print(f"The point is WITHIN the point cloud ({1 - prob:.2f})")

The implementation of this method is somewhat less robust as no SDF sampling is performed. Instead, only the original points comprising the point cloud that represents the surface are used. This makes this method faster to execute and easier to implement, however there is no guarantee for convergence for complex boundary shapes as it is very sensitive on the number of examined closest points on the surface.

In [ ]:
def assess_position_2(
    query_points: np.ndarray,
    evaluation_points: np.ndarray,
    sample_count: int = 5,
    normals: np.ndarray | None = None,
    k: int | None = None,
) -> np.ndarray:
    """Return the indicator stating whether the query point is out of the surface.

    Args:
        query_points (numpy.ndarray): Mx3 array of query point coordinates.
        evaluation_points (numpy.ndarray): Nx3 array of surface point coordinates.
        sample_count (int, optional): Number of samples to consider for the decision.
        normals (numpy.ndarray, optional): Nx3 array of surface point normals.
        k (int, optional): Number of neighbors for normal estimation and orientation.

    Returns:
        numpy.ndarray: Boolean M-sized array indicating if each query point is outside.

    """
    # Handle points
    size = evaluation_points.shape[0]
    if size < MIN_SIZE:
        msg = f"Number of points must be > {MIN_SIZE}"
        raise ValueError(msg)  # For robustness

    # Compute normals
    if normals is None:
        if not k:
            k = int(2 * np.log(size))
            k = max(5, min(k, 30))  # Constrain k between 5 and 30
        normals = estimate_normals(evaluation_points, k)
        normals = orient_normals(evaluation_points, normals, k)
    normals = normals / np.linalg.norm(normals, axis=1).reshape(-1, 1)

    # Find points on the surface closest to the query point
    tree = spatial.KDTree(evaluation_points)
    _, idx = tree.query(query_points, k=sample_count, workers=-1)
    closest_points = evaluation_points[idx]

    # Compute the dot product between the relative position and normal vector
    pos_vec = np.atleast_2d(query_points)[:, np.newaxis, :] - closest_points

    # Count the positive values indicating that the point is located outside
    out = np.einsum("ijk,ijk->ij", pos_vec, normals[idx]) > 0
    return np.sum(out, axis=1) >= sample_count * 0.5

In [ ]:
# Define query points to test an assessment function
query_points = np.c_[X.ravel(), Y.ravel(), np.ones_like(X).ravel()]

In [ ]:
# Plot query points in 3D
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax.contourf(X, Y, Z, zdir="y", offset=1, levels=1, colors="b")
ax.contourf(X, Y, Z, zdir="x", offset=-1, levels=1, colors="b")
ax.scatter(*query_points.T, fc="orange", ec="k", s=5, lw=0.5)
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# Check if the orange dots are inside or out of the point cloud
out = assess_position_2(query_points, points, sample_count=5, normals=normals)

In [ ]:
# Plot query points in 3D with assessment
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax.contourf(X, Y, Z, zdir="y", offset=1, levels=1, colors="b")
ax.contourf(X, Y, Z, zdir="x", offset=-1, levels=1, colors="b")
ax.scatter(*query_points[out, ...].T, fc="w", ec="k", s=15, lw=0.5)
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# Find out the approximate radius where f(x, y) is ~1
idx = np.where(np.isclose(Z, 1, rtol=1e-2, atol=1e-2))
r = np.mean(np.sqrt(X[idx] ** 2 + Y[idx] ** 2))

In [ ]:
# Plot only the xy-projection of the outside points
fig = plt.figure(figsize=(4, 4))
ax = plt.axes()
circle = patches.Circle((0, 0), r, fc="none", ec="k")
ax.add_patch(circle)
ax.scatter(*query_points[out, :2].T, fc="w", ec="k", s=7, lw=0.5)
ax.set(xlabel="x", ylabel="y");

# Extraction of the points on the boundary of the point cloud

Let’s assume we have a set of points, $\mathbb{X} = \{\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n\}$, where $\mathbf{x}_i = (x_i, y_i, z_i)$ with $1 \leq i \leq n$, sampling a compact region $\Omega \subset \mathbb{R}^3$. We want to identify the subset of points that lie on the boundary surface $S = \partial \Omega$, called *surface points*.

<div style="text-align:center">
    <img style="margin:20px; width:450px;" src="media/pc-surf.svg">
</div>

The following steps should be applied to each point, $\mathbf{x}_i$, in $\mathbb{X}$.

<div style="text-align:center">
    <img style="margin:20px; width:750px;" src="media/pc-surf-extract.svg">
</div>

The following is the simple implementation in Python by using only the `SciPy.spatial` module.

In [ ]:
def extract_surface_points(points: np.ndarray, radius: float) -> np.ndarray:
    """Return surface points on the point cloud.

    Args:
        points (numpy.ndarray): Nx3 array of point coordinates.
        radius (float): Radius to define local neighborhoods.

    Returns:
        numpy.ndarray: Mx3 array of surface point coordinates.

    """
    surface_points = []
    tree = spatial.KDTree(points)
    for point in tqdm(points):
        # Step 1: extract a local neighbourhood around the query point
        idx = tree.query_ball_point(point, r=radius, eps=0, p=2)
        nbh = points[idx]

        # Step 2: estimate normal direction at the query point
        x = nbh.copy()
        x = x - np.mean(x, axis=0)
        c = x.T @ x
        u, s, _ = np.linalg.svd(c)
        n = u[:, np.argmin(s)]

        # Step 3: search two circular patches within neighbourhood
        centers = [point + n * radius / 2, point - n * radius / 2]
        for center in centers:
            ii = tree.query_ball_point(center, r=radius / 2, eps=0, p=2)
            if len(ii) in [0, 1]:
                surface_points.append(point)

    return np.unique(surface_points, axis=0)

Toy-problem example: extraction of the surface points on the spherical surface of the ball of radius 1 with uniformly distributed points within the volume.

In [ ]:
def _ball(num_points: int) -> np.ndarray:
    points = []
    for _ in range(num_points):
        # Random radius with cubic root for uniform distribution inside a sphere
        r = RNG.uniform() ** (1 / 3)
        # Random spherical coordinates
        theta = RNG.uniform(0, 2 * np.pi)
        phi = RNG.uniform(0, np.pi)
        # Convert spherical coordinates to Cartesian coordinates
        x = r * np.sin(phi) * np.cos(theta)
        y = r * np.sin(phi) * np.sin(theta)
        z = r * np.cos(phi)
        points.append([x, y, z])
    return np.array(points)

In [ ]:
# Generate points withina ball of radius 1
points = _ball(9999)

In [ ]:
# Show the original point cloud
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax.scatter(*points.T, s=1, alpha=0.25)
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# Extract the surface points
surface_points = extract_surface_points(points, radius=0.5)

In [ ]:
# Show the extracted surface points
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection="3d")
ax.scatter(*surface_points.T, s=1)
ax = set_3d_params(ax)
ax.view_init(25, -70);